# Byte-pair encoding

Че получается: 
- Есть две крайности кодирование текста как последовательности символов и как последовательности токенов. Оба хуже.
- Можно брать просто n-граммы и подавать их как символы (долго), или например усреднять их вектора (fastText - (для неизвестных слов вектор строится как наиболее близкий по символьным эмбеддингам)) и но это слишком абстрактный подход, для генерации текста не очень подходит
- Промежуточный вариант - учет n-грамм в составе последовательностей по частоте (**byte pair encoding**) 
  - "aabc aabaa" -> "Xbc XbX -> "Yc YX"

Это позволяет сильно сократить размер словаря и, при этом, не сильно увеличить длину входных цепочек токенов, в отличие от символьных языковых моделей.

Нужно отметить, что пары имеет смысл выделять только в рамках токена, поэтому в пределе алгоритм просто выходит на кодирование отдельных токенов целиком.
- получается, что это промежуточный вариант, начинается с символьного кодирования и итеративно доходит до токенов
- поэтому надо где-то остановиться, 
- то есть глубина кодирования это еще один гиперпараметр модели, который надо подбирать
  - некоторый процент от разности количества слов (максимум объема кодирующего словаря) и количества символов (минимум). Можно добавить, что это максимум и минимум, если различных слов в корпусе больше, чем различных символов, остальное можно считать вырожденными случаями в рамках таких задач, иначе зачем было вообще алфавит придумывать, если не так, то могут быть ньюансы (интересно, как этот момент с иероглифами пересекается).
- данный гиперпараметр идентичен гиперпараметру - размер словаря модели
- добавлены токены для начала слова и конца слова `<w>` и `</w>`, начала и конца абзаца `<n>` и `</n>`

In [30]:
import re
from collections import defaultdict
from collections import Counter

import matplotlib.pyplot as plt


text1 = """Че получается: 
- Есть две крайности кодирование текста как последовательности символов и как последовательности токенов. Оба хуже.
- Можно брать просто n-граммы и подавать их как символы (долго), или например усреднять их вектора (fastText - (для неизвестных слов вектор строится как наиболее близкий по символьным эмбеддингам)) и но это слишком абстрактный подход, для генерации текста не очень подходит
- Промежуточный вариант - учет n-грамм в составе последовательностей по частоте (**byte pair encoding**) 
  - "aabc aabaa" -> "Xbc XbX -> "Yc YX"

Это позволяет сильно сократить размер словаря и, при этом, не сильно увеличить длину входных цепочек токенов, в отличие от символьных языковых моделей.

Нужно отметить, что пары имеет смысл выделять только в рамках токена, поэтому в пределе алгоритм просто выходит на кодирование отдельных токенов целиком.
- получается, что это промежуточный вариант, начинается с символьного кодирования и итеративно доходит до токенов
- поэтому надо где-то остановиться, 
- то есть глубина кодирования это еще один гиперпараметр модели, который надо подбирать"""

text = "There is an 80% chance of rainfall today. We are pretty sure it is going to rain."

# get the word frequency and add the end of word (</w>) token at the end of each word
def make_word_dict(text):
    """возвращает частоты токентов в тексте"""
    words = ["<w>" + ' '.join(token) + "</w>" for token in text.strip().split(" ")]
    return Counter(words)

def get_pairs(word_freq_dict):
    """возвращает биграммы в токенах и их частоты"""
    pairs = defaultdict(int)
    for word, freq in word_freq_dict.items():
        chars = word.split()
        for i in range(len(chars)-1):
            pairs[chars[i], chars[i+1]] += freq
    return pairs

def merge_byte_pairs(best_pair, word_freq_dict):
    """возвращает копию словаря с объединенными в подстроку биграммами"""
    merged_dict = {}
    bigram = re.escape(' '.join(best_pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)') # все такие биграммы, обрамленные пробелами
    for word in word_freq_dict:
        w_out = p.sub(''.join(best_pair), word)     # убрать из биграммы пробел (объединение)
        merged_dict[w_out] = word_freq_dict[word]
    return merged_dict

def get_subword_tokens(word_freq_dict):
    """возвращает словарь частот подстрок"""
    char_freq_dict = defaultdict(int)
    for word, freq in word_freq_dict.items():
        chars = word.split()
        for char in chars:
            char_freq_dict[char] += freq
    return char_freq_dict

def train(word_freq_dict, rate=0.5):
    """
    Возвращает список субтокенов заданной длины:
        rate - целевой размер кодирующего словаря как доля от исходного количества токенов
        rate = 1 (кодирование токенов целиком), rate = 0 (символьное кодирование)
    """
    init_len = len(word_freq_dict)
    temp_word_freq_dict = word_freq_dict.copy()
    subword_tokens = get_subword_tokens(temp_word_freq_dict)

    while len(subword_tokens) / init_len <= rate:
        pairs = get_pairs(temp_word_freq_dict)
        if not pairs:
            break
        best_pair = max(pairs, key=pairs.get)
        temp_word_freq_dict = merge_byte_pairs(best_pair, temp_word_freq_dict)
        subword_tokens = get_subword_tokens(temp_word_freq_dict)
    
    return subword_tokens

word_freq_dict = make_word_dict(text)
new_freq_dict1 = train(word_freq_dict, rate=1)
new_freq_dict0 = train(word_freq_dict, rate=0)
len(new_freq_dict0), len(new_freq_dict1)
# new_freq_dict0

(38, 38)

In [18]:
from bpe import Encoder     # trash

# Generated with http://pythonpsum.com
test_corpus = '''
    Object\nraspberrypi functools dict kwargs. Gevent raspberrypi functools. Dunder raspberrypi decorator dict didn't lambda zip import pyramid, she lambda iterate?
    Kwargs raspberrypi diversity unit object gevent. Import fall integration decorator unit django yield functools twisted. Dunder integration decorator he she future. Python raspberrypi community pypy. Kwargs integration beautiful test reduce gil python closure. Gevent he integration generator fall test kwargs raise didn't visor he itertools...
    Reduce integration coroutine bdfl he python. Cython didn't integration while beautiful list python didn't nit!
    Object fall diversity 2to3 dunder script. Python fall for: integration exception dict kwargs dunder pycon. Import raspberrypi beautiful test import six web. Future integration mercurial self script web. Return raspberrypi community test she stable.
    Django raspberrypi mercurial unit import yield raspberrypi visual rocksdahouse. Dunder raspberrypi mercurial list reduce class test scipy helmet zip?
'''

encoder = Encoder(200, pct_bpe=0.88)  # params chosen for demonstration purposes
encoder.fit(test_corpus.split('\n'))

example = "Vizzini: He didn't fall? INCONCEIVABLE!"
print(encoder.tokenize(example))
# ['__sow', 'vi', 'z', 'zi', 'ni', '__eow', '__sow', ':', '__eow', 'he', 'didn', "'", 't', 'fall', '__sow', '?', '__eow', '__sow', 'in', 'co', 'n', 'ce', 'iv', 'ab', 'le', '__eow', '__sow', '!', '__eow']
print(next(encoder.transform([example])))
# [24, 108, 82, 83, 71, 25, 24, 154, 25, 14, 10, 11, 12, 13, 24, 85, 25, 24, 140, 59, 39, 157, 87, 165, 114, 25, 24, 148, 25]
print(next(encoder.inverse_transform(encoder.transform([example]))))
# vizzini : he didn ' t fall ? inconceivable !

['__sow', 'vi', 'z', 'zi', 'ni', '__eow', '__sow', ':', '__eow', 'he', 'didn', "'", 't', 'fall', '__sow', '?', '__eow', '__sow', 'in', 'co', 'n', 'ce', 'iv', 'ab', 'le', '__eow', '__sow', '!', '__eow']
[24, 108, 82, 83, 71, 25, 24, 154, 25, 14, 10, 11, 12, 13, 24, 85, 25, 24, 140, 59, 39, 157, 87, 165, 114, 25, 24, 148, 25]
vizzini : he didn ' t fall ? inconceivable !


In [41]:
# from tokenizers import CharBPETokenizer
from tokenizers.pre_tokenizers import Metaspace, Whitespace, Sequence

from tokenizers import Tokenizer, normalizers, decoders, models

tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizers = Sequence([Metaspace("\n"), Whitespace()])

tokenizer.train_from_iterator(test_corpus)
print(tokenizer.get_vocab_size(), len(tokenizer.get_vocab()), max(tokenizer.get_vocab().values()))  # OK

coded = tokenizer.encode("unit object\ngevent")
coded.tokens, coded.ids, tokenizer.decode(coded.ids)
tokenizer.get_vocab()




44 44 43


{',': 4,
 'f': 24,
 'b': 20,
 'g': 25,
 ' ': 1,
 'R': 18,
 'e': 23,
 '?': 9,
 'i': 27,
 'v': 39,
 'I': 14,
 'p': 34,
 'a': 19,
 '2': 6,
 '\n': 0,
 'l': 30,
 'h': 26,
 "'": 3,
 's': 36,
 'm': 31,
 '.': 5,
 ':': 8,
 'F': 12,
 'k': 29,
 '!': 2,
 'c': 21,
 'd': 22,
 'w': 40,
 'P': 17,
 'C': 10,
 'z': 43,
 'y': 42,
 'G': 13,
 'u': 38,
 'r': 35,
 'j': 28,
 'D': 11,
 'K': 15,
 't': 37,
 '3': 7,
 'o': 33,
 'O': 16,
 'n': 32,
 'x': 41}